# Data Processing Libraries Comparison

This notebook compares different data processing libraries: Pandas, Polars, DuckDB, and PySpark.

## Configuration and Imports

First, we'll set up our environment and import all necessary libraries.

In [7]:
# Set the file path
DATA_FILE = '../files/sample_ads_data.csv'

# Import all required libraries
import pandas as pd
import polars as pl
import duckdb
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Performance measurement imports
import time
import psutil
import os

# Initialize PySpark
spark = SparkSession.builder.getOrCreate()

# Helper function for performance measurement
def measure_performance(func):
    """Measure execution time and memory usage of a function"""
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / 1024 / 1024  # Memory in MB
    start_time = time.time()
    
    result = func()
    
    end_time = time.time()
    end_memory = process.memory_info().rss / 1024 / 1024
    
    return {
        'execution_time': end_time - start_time,
        'memory_used': end_memory - start_memory,
        'result': result
    }

# Performance Comparisons

Let's compare the performance of different operations across all four libraries.

## 1. Data Loading Performance
Compare how fast each library loads the CSV file and how much memory it uses.

In [5]:
# Test data loading for each library
results = {}

# Pandas
results['pandas'] = measure_performance(
    lambda: pd.read_csv(DATA_FILE)
)
df = results['pandas']['result']

# Polars (force eager evaluation)
results['polars'] = measure_performance(
    lambda: pl.read_csv(DATA_FILE)
)
df_pl = results['polars']['result']

# DuckDB (already eager, but ensure result is materialized)
results['duckdb'] = measure_performance(
    lambda: duckdb.query(f"SELECT * FROM read_csv('{DATA_FILE}', quote='\"')").df()
)
df_duck = results['duckdb']['result']

# PySpark (convert to pandas to force full load in memory)
results['pyspark'] = measure_performance(
    lambda: spark.read.csv(DATA_FILE, header=True, inferSchema=True)
)
df_spark = results['pyspark']['result']

# Display results
pd.DataFrame({
    'Library': list(results.keys()),
    'Loading Time (s)': [r['execution_time'] for r in results.values()],
    'Memory Used (MB)': [r['memory_used'] for r in results.values()]
}).round(3)

,Library,Loading Time (s),Memory Used (MB)
0,pandas,20.380,1642.121
1,polars,2.119,1259.648
2,duckdb,16.334,-2856.938
3,pyspark,4.345,1.699


## 2. Filtering Performance
Compare how fast each library can filter data based on a condition.

In [10]:
# Test filtering performance
filter_results = {}

# Pandas
filter_results['pandas'] = measure_performance(
    lambda: df[df['account_id'] == 'ACC0001'].shape[0]
)
# Polars
filter_results['polars'] = measure_performance(
    lambda: df_pl.filter(pl.col('account_id') == 'ACC0001').shape[0]
)

# DuckDB
filter_results['duckdb'] = measure_performance(
    lambda: duckdb.query("SELECT count(*) FROM df_duck WHERE account_id = 'ACC0001'").df().iloc[0, 0]
)

# PySpark
filter_results['pyspark'] = measure_performance(
    lambda: df_spark.filter(df_spark['account_id'] == 'ACC0001').count()
)

# Print counts for each library
print("Pandas count:", filter_results['pandas']['result'])
print("Polars count:", filter_results['polars']['result'])
print("DuckDB count:", filter_results['duckdb']['result'])
print("PySpark count:", filter_results['pyspark']['result'])

# Display results
pd.DataFrame({
    'Library': list(filter_results.keys()),
    'Filtering Time (s)': [r['execution_time'] for r in filter_results.values()],
    'Memory Used (MB)': [r['memory_used'] for r in filter_results.values()]
}).round(3)

Pandas count: 249602
Polars count: 249602
DuckDB count: 249602
PySpark count: 249602


,Library,Filtering Time (s),Memory Used (MB)
0,pandas,4.506,1621.137
1,polars,0.150,43.359
2,duckdb,0.025,0.559
3,pyspark,1.138,0.113


## 3. GroupBy and Aggregation Performance
Compare how fast each library can perform groupby operations and aggregations.

In [9]:
# Test groupby and aggregation performance
groupby_results = {}

# Pandas
groupby_results['pandas'] = measure_performance(
    lambda: df.groupby('account_id')['campaign_id'].count()
)

# Polars
groupby_results['polars'] = measure_performance(
    lambda: df_pl.group_by('account_id').agg(pl.col('campaign_id').count())
)

# DuckDB
groupby_results['duckdb'] = measure_performance(
    lambda: duckdb.query("SELECT account_id, COUNT(campaign_id) as count FROM df_duck GROUP BY account_id")
)

# PySpark
groupby_results['pyspark'] = measure_performance(
    lambda: df_spark.groupBy('account_id').count().toPandas()
)

# Display results
pd.DataFrame({
    'Library': list(groupby_results.keys()),
    'GroupBy Time (s)': [r['execution_time'] for r in groupby_results.values()],
    'Memory Used (MB)': [r['memory_used'] for r in groupby_results.values()]
}).round(3)

,Library,GroupBy Time (s),Memory Used (MB)
0,pandas,2.408,78.668
1,polars,1.475,2235.391
2,duckdb,0.067,2.453
3,pyspark,5.102,1.637


## 4. Add new column Performance
Compare how fast each library can add a new column to the DataFrame.

In [ ]:
# Compare performance of adding a new column with default value
addcol_results = {}

# Pandas
addcol_results['pandas'] = measure_performance(
    lambda: df.assign(NewCol='default')
)

# Polars
addcol_results['polars'] = measure_performance(
    lambda: df_pl.with_columns([pl.lit('default').alias('NewCol')])
)

# DuckDB
addcol_results['duckdb'] = measure_performance(
    lambda: duckdb.query("SELECT *, 'default' AS NewCol FROM df_duck")
)

# PySpark
addcol_results['pyspark'] = measure_performance(
    lambda: df_spark.withColumn('NewCol', F.lit('default'))
)

# Display results
pd.DataFrame({
    'Library': list(addcol_results.keys()),
    'AddCol Time (s)': [r['execution_time'] for r in addcol_results.values()],
    'Memory Used (MB)': [r['memory_used'] for r in addcol_results.values()]
}).round(3)

,Library,AddCol Time (s),Memory Used (MB)
0,pandas,0.012,34.43
1,polars,0.004,3.43
2,duckdb,0.005,0.02
3,pyspark,0.007,0.00


## 5. Hash Operation Performance
Compare how fast each library can compute a hash of the 'VIN' column.

In [15]:
# Compare performance of hashing the 'County' column
hash_results = {}

# Pandas
import hashlib
hash_results['pandas'] = measure_performance(
    lambda: df['date'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
)

# Polars
hash_results['polars'] = measure_performance(
    lambda: df_pl.with_columns([
        df_pl.get_column('date').map_elements(lambda x: hashlib.sha256(str(x).encode()).hexdigest(), return_dtype=pl.String).alias('date_hash')
    ])
)

# DuckDB
hash_results['duckdb'] = measure_performance(
    lambda: duckdb.query("SELECT *, sha256(CAST(date as VARCHAR)) AS date_hash FROM df_duck")
)

# PySpark
hash_results['pyspark'] = measure_performance(
    lambda: df_spark.withColumn('date_hash', F.sha2(F.col('date').cast('string'), 256))
)

# Display results
pd.DataFrame({
    'Library': list(hash_results.keys()),
    'Hash Time (s)': [r['execution_time'] for r in hash_results.values()],
    'Memory Used (MB)': [r['memory_used'] for r in hash_results.values()]
}).round(3)

,Library,Hash Time (s),Memory Used (MB)
0,pandas,3.917,283.676
1,polars,4.371,461.555
2,duckdb,0.059,81.250
3,pyspark,0.009,0.000


## Conclusions

After running these comparisons, you can evaluate each library based on:

1. **Performance**
   - Loading speed
   - Memory usage
   - Operation speed (filtering, groupby)

2. **Syntax**
   - Code readability
   - API consistency
   - Learning curve

3. **Use Cases**
   - Pandas: Best for small to medium datasets, interactive analysis
   - Polars: Great for large datasets on a single machine
   - DuckDB: Excellent for SQL users and complex queries
   - PySpark: Ideal for distributed processing and very large datasets

Choose the library that best fits your specific needs based on:
- Dataset size
- Performance requirements
- Team expertise
- Integration needs